Function to transform the raw data as explained in the preprocessing notebook.

In [1]:
def change_origin(X):
    
    x0 = X['x0'].copy()
    y0 = X['y0'].copy()
    counter = 0
    
    for column in X.columns:
        
        if counter%2 == 0:
            X.loc[:,column] -= x0 #loc is faster, it computes the whole column at once instead of one row at a time
        else:
            X.loc[:,column] -= y0
        counter +=  1

    return X

def separate_x_y(pos):
    
    x = [] 
    y = [] 
    counter =0
    
    for i in pos:
        if counter%2==0:
            x.append(i)
        else: y.append(i)
        counter += 1
        
    return x, y

def normalize_pos(pos):
    
    x_coords_list, y_coords_list = separate_x_y(pos)
    max_x = max(x_coords_list)
    min_x = min(x_coords_list)
    max_y = max(y_coords_list)
    min_y = min(y_coords_list)
    
    for i in range(len(pos)):
        if i%2==0:
            pos[i] = (pos[i]- min_x)/(max_x-min_x)
        else:
            pos[i] = (pos[i]- min_y)/(max_y-min_y)    
        
    return pos

def normalize_dataset(X):
    
    X_normalized = X.copy()
    
    for i in range(len(X)):
        X_normalized.iloc[i] = normalize_pos(X.iloc[i])
        
    return X_normalized

def transform_fn(raw_X_df):
    X_normalized = normalize_dataset(raw_X_df).copy()
    X_normalized_and_centered = change_origin(X_normalized).copy()
    transformed_X_df = X_normalized_and_centered.iloc[:, 2:].copy()
    return transformed_X_df


Hand Detector Object:

In [2]:
import mediapipe as mp
import cv2
import time
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import math
import os
import nbconvert

class hand_detector():
    def __init__(self, static=False, n_maxhands=1, detectionCon=0.5, trackingCon=0.5):
        self.mode = static
        self.n_maxhands = n_maxhands
        self.detectkionCon = detectionCon
        self.trackingCon = trackingCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.n_maxhands, self.detectkionCon, self.trackingCon)
        self.mpDraw = mp.solutions.drawing_utils
        
    def find_hands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        self.results = self.hands.process(imgRGB) #The model processes the img, results has all the information, ALL. 
        #print(self.results.multi_hand_landmarks) #multi hand landmarks have the coordenates of all the nodes that define a hand for the model. 
        if self.results.multi_hand_landmarks and draw:
            for handLms in self.results.multi_hand_landmarks: #for each hand
                self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS) #this is a single hand. We dont want it to write it in the RGB image, bc the img that we are 
                #displaying is img not imgRGB
                ##now we want to draw the line that  connects all those dots for each hand

        return img
    def find_postion(self, img, handNo=0, draw=True ):
        if self.results.multi_hand_landmarks:
            hand = self.results.multi_hand_landmarks[handNo]
            lmList = []
            h, w, c = img.shape #height, width and channels of img
            for id_,lm in enumerate(hand.landmark):
                #the coordenates are scaled, [0,1] not in pixels, to transform to pixels
                cx, cy = int(lm.x*w), int(lm.y*h)
                lmList.append([id_, cx, cy])
                #if id_ == 0:
                #cv2.circle(img, (cx, cy), 15,(255,0,255), cv2.FILLED )
            return lmList
        else: return 'None'

Function that runs the Main Program:

In [3]:
import pickle
import mediapipe as mp
import cv2
import time
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import math
import os
import nbconvert

def sample(pos):
    row = []
    for coords in pos:
        #print(coords)
        row.append(coords[1])
        row.append(coords[2])
    return row

def coords_change(row):

    counter = 0
    x0 = row[counter]
    y0 = row[counter+1]
    position = []
    for i in row:
    
        if counter%2 == 0:
            i = i - x0
            position.append(i)
        else:
            i = i - y0
            position.append(i)
        counter += counter + 1
    
    return position

def main_fn(filename):
    
    model =  pickle.load(open(filename, 'rb'))

    detector = hand_detector()
    
    cap = cv2.VideoCapture(0) #opens camera for videocapture

    ptime = 0
    itime = time.time()
    letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 
               'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
    while True:
        success, img =  cap.read() # this will give us our frame
        img = cv2.flip(img,1)
        img = detector.find_hands(img)      
        position = detector.find_postion(img)
        
        if position!='None':
            
            result_coords =[position[0][1], position[0][2]]
            position = sample(position)
            position = normalize_pos(position)
            position = coords_change(position)
            position = np.array(position[2:]).reshape(1,-1)
            
            '''Prediction'''
            y_pred = model.predict(position)
            y_pred = letters[int(y_pred)-1]
        else: pass
        
        ctime = time.time()
        fps = 1/(ctime-ptime)
        ptime = ctime
        timer = ctime-itime
        
        cv2.putText(img, str(int(fps)), (10,40), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1,  (255,0,255), 3)
        cv2.putText(img, str(int(timer)), (600,40), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1,  (255,0,255), 3)
        if position!='None':
            coords = (result_coords[0]-10, result_coords[1]+40)
            cv2.putText(img, f'{y_pred}', coords, cv2.FONT_HERSHEY_SIMPLEX,1,  (255,0,255), 3)
        cv2.imshow('Image', img) 
        cv2.waitKey(1)